In [ ]:
import matplotlib.pyplot as plt
import librosa.display
import os
import numpy as np
import pandas as pd
import librosa
from pydub import AudioSegment
import math
import random
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import csv
import shutil



In [ ]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 1000
        t2 = to_min  * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export('./data/splitMusic' + '/' + split_filename, format="wav")
        
    def multiple_split(self, sec_per_split, start_sec, end_bef, rand_amount):
        total_mins = math.floor(self.get_duration())
        #print(total_mins)
        for i in range(rand_amount):
            rand_start = random.randint(start_sec, total_mins-end_bef)
            split_fn = self.filename.replace('.wav','') + '_' + str(rand_start) +'.wav'
            self.single_split(rand_start, rand_start+sec_per_split, split_fn)
            print(str(i) + ' Done')
            #if i == total_mins - min_per_split:
                #print('All splited successfully')

In [ ]:
def plot_spectrogram(Y, sr, hop_length, filename, y_axis="log"):
    fig = plt.figure(figsize=(25, 10))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    librosa.display.specshow(Y, sr=sr, hop_length=hop_length, y_axis=y_axis)
    ax.axes.yaxis.set_visible(False)

    fig.savefig("./images/"+filename, bbox_inches='tight', pad_inches=0)

In [ ]:
def createCSV():
    header = ['name', 'bpm']
    lst = os.listdir("./data/images")
    #data = convert(header, 0, lst)
    #print(data)
    with open('data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        totLst = []
        for i in range (len(lst)):
            tmpLst = []
            tmpLst.append(lst[i][4:])
            tmpLst.append(lst[i][0:3]) 
            totLst.append(tmpLst)
            print(tmpLst)
            writer.writerows([tmpLst])


In [ ]:
def sortImgs():
    path = '.'
    lst = os.listdir("./images")
    if not os.path.exists(path+"/data"):
        os.makedirs(path+'/data')
    for fle in lst:
        bpm = fle[0:3]
        if os.path.exists(path+"/data/"+bpm):
            shutil.copy(path+'/images/'+fle, path+'/data/'+bpm+'/'+fle)
        else:
            os.makedirs(path+'/data/'+bpm)
            shutil.copy(path+'/images/'+fle, path+'/data/'+bpm+'/'+fle)

In [ ]:
#function to splice a wav file into random multiple segments
mylist = os.listdir("./music")
print(mylist)
#print (len(mylist))
FRAME_SIZE = 2048
HOP_SIZE = 512

for x in mylist:
    print(x)
    folder = './music'
    file = x
    split_wav = SplitWavAudioMubin(folder, file)
    split_wav.multiple_split(sec_per_split=6, start_sec=60,  end_bef=60, rand_amount=10)

In [ ]:
mylist = os.listdir("./splitMusic")
for x in mylist:
    filename = x.replace('.wav','')
    filelocation = "./splitMusic/"+ x
    scale, sr = librosa.load(filelocation)
    S_scale = librosa.stft(scale, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    Y_scale = np.abs(S_scale) ** 2
    Y_scale.shape
    Y_log_scale = librosa.power_to_db(Y_scale)
    plot_spectrogram(Y_log_scale, sr, HOP_SIZE, filename, y_axis="log")


In [ ]:
createCSV()

In [ ]:
sortImgs()